In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import autokeras as ak
import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.utils import to_categorical

from sklearn import preprocessing
from sklearn.svm import SVC 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score,train_test_split, RepeatedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier


import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

In [2]:
rootPath = "C:\\Users\\mlb\\Teste_VOCs\\1_Preprocessing\\NewDatasets"
columnsName_X = ["TGS826","TGS2611","TGS2603","TGS813","TGS822","TGS2602","TGS823","temperatura","pressao","umidade","culture_day"]

In [3]:
AllCandidasCollect    = pd.read_csv(rootPath+"\\AllCandidasCollect.csv", sep=",")
AllCandidasCollect

,TGS826,TGS2611,TGS2603,TGS813,TGS822,TGS2602,TGS823,temperatura,pressao,umidade,c_type,culture_day
0,91720.0,7305.0,6014.0,63460.0,4294.0,40700.0,8005.0,60.66,100.22,12.59,1,1
1,91910.0,7305.0,6020.0,63380.0,4293.0,40730.0,7998.0,60.71,100.15,12.52,1,1
2,91190.0,7305.0,6020.0,63460.0,4294.0,40750.0,7998.0,60.70,100.19,12.56,1,1
3,92380.0,7305.0,6020.0,64470.0,4319.0,40770.0,7998.0,60.69,83.83,12.60,1,1
4,35120.0,7305.0,7841.0,64730.0,2047.0,16630.0,7032.0,60.62,74.81,11.56,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
64961,87790.0,7171.0,8935.0,38260.0,2307.0,15180.0,7706.0,57.33,54.08,7.23,6,2
64962,86770.0,7171.0,8941.0,38200.0,2305.0,15180.0,7710.0,57.32,54.00,7.25,6,2
64963,87190.0,7171.0,8941.0,38260.0,2307.0,15180.0,7704.0,57.30,54.08,7.40,6,2
64964,87190.0,7171.0,8941.0,38330.0,2297.0,15170.0,7706.0,57.29,54.08,7.25,6,2


In [4]:
X = AllCandidasCollect.drop(columns=['c_type'])
y = AllCandidasCollect['c_type']
y_sorted , y_description = pd.factorize(AllCandidasCollect['c_type'], sort=False)
categorical_labels_y = to_categorical(y_sorted)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=2)  

In [6]:
def getMetrics(y_true, y_pred):
    precision, recall, fbeta_score, support = precision_recall_fscore_support(y_true, y_pred, average='micro')
    f_measure = 2* ((precision*recall)/(precision+recall))
    return   precision, recall, (1-precision), f_measure

In [7]:
def trainingWithKfoldsModel(skf, model, X_rest, y_rest, isVal):
    i = 0
    treiningScore = []
    predictions_score = []
    
    precision_score =[]
    recall_score = []
    error_score = []
    f_measure_score = []
    
    if (isVal):
        treiningScore.append(model.fit(X_rest,y_rest).score(X_rest,y_rest))
        y_pred = model.predict(X_val)
        predictions_score.append(accuracy_score(y_pred, y_val ))
        a, b , c , d = getMetrics(y_pred, y_val)
        precision_score.append(a)
        recall_score.append(b)
        error_score.append(c)
        f_measure_score.append(d)
    else:
        for train_index, test_index in skf.split(X_rest, y_rest):
            print(f"Iteração: {i} do modelo")
            X_train, X_test = X_rest.iloc[train_index], X_rest.iloc[test_index]
            y_train, y_test = y_rest.iloc[train_index], y_rest.iloc[test_index] 
            #print(f"XTrain:{X_train}")
            treiningScore.append(model.fit(X_train,y_train).score(X_train,y_train))
            y_pred = model.predict(X_test)
            predictions_score.append(accuracy_score(y_pred, y_test))
            a, b , c , d = getMetrics(y_pred, y_test)
            precision_score.append(a)
            recall_score.append(b)
            error_score.append(c)
            f_measure_score.append(d)
            
            i=i+1
    return [treiningScore, predictions_score
            ,np.mean(precision_score),np.mean(recall_score)
            ,np.mean(error_score),np.mean(f_measure_score)
            , model]

In [8]:
rkf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=15)

treiningScore = []
predictions_score = []

array_score_test = []
array_score_train = []
array_std_test = []
array_std_train = []

precision_score =[]
recall_score = []
error_score = []
f_measure_score = []

In [9]:
def trein_model(cls, N):
    max_acc = 0
    model_f = None 
    for i in range(N):
        print(f"Iteração {i}")
        treinScore, predictionScore, p, r, e , f, model =  trainingWithKfoldsModel(rkf, cls, X_train, y_train, False) 

        array_score_train.append(np.mean(treinScore))
        array_score_test.append(np.mean(predictionScore))
        array_std_test.append(np.std(treinScore))
        array_std_train.append(np.std(predictionScore))
        precision_score.append(p)
        recall_score.append(r)
        error_score.append(e)
        f_measure_score.append(f)

        if(max_acc<np.mean(predictionScore)):
            print(f"Max acc: {np.mean(predictionScore)}")
            max_acc = np.mean(predictionScore)
            model_f = model

    print(f"Resultado para a {cls}: Score  train:{np.mean(array_score_train)}")
    print(f"Resultado para a {cls}: Score  test:{np.mean(array_score_test)}")
    print(f"Resultado para a {cls}: Desvio train:{np.mean(array_std_train)}")
    print(f"Resultado para a {cls}: Desvio test:{np.mean(array_std_test)}")

    print(f"Resultado para a {cls}: precision test:{np.mean(precision_score)}")
    print(f"Resultado para a {cls}: recall    test:{np.mean(recall_score)}")
    print(f"Resultado para a {cls}: error     test:{np.mean(error_score)}")
    print(f"Resultado para a {cls}: f_measure test:{np.mean(f_measure_score)}")
    return model_f

In [10]:
#mlp = MLPClassifier(random_state=5, max_iter=500, alpha=1e-5, hidden_layer_sizes=(128,), activation='relu')
#model_f = trein_model(mlp, 10 )
#model_f

In [11]:
#treinScore, predictionScore, p, r, e , f, model =  trainingWithKfoldsModel(rkf, model_f, X_train, y_train, True) 

In [12]:
# print(f"Resultado para a MLP: Score  train:{np.mean(treinScore)}")
# print(f"Resultado para a MLP: Score  test:{np.mean(predictionScore)}")
# print(f"Resultado para a MLP: Desvio train:{np.std(treinScore)}")
# print(f"Resultado para a MLP: Desvio test:{np.std(predictionScore)}")

# print(f"Resultado para a MLP: precision test:{np.mean(p)}")
# print(f"Resultado para a MLP: recall    test:{np.mean(r)}")
# print(f"Resultado para a MLP: error     test:{np.mean(e)}")
# print(f"Resultado para a MLP: f_measure test:{np.mean(f)}")

In [13]:
#neigh = KNeighborsClassifier(n_neighbors=3)
#model_f_neigh= trein_model(neigh, 10 )
#model_f_neigh

In [14]:
#etc = ExtraTreesClassifier(bootstrap=False,max_features=0.9500000000000001, min_samples_leaf=6, min_samples_split=3, n_estimators=100)
#model_etc_f = trein_model(etc, 10 )
#model_etc_f

In [ ]:
kernels =  ['rbf', 'poly', 'sigmoid','linear']
C = [0.1,1, 10, 100]
gamma = [1,0.1,0.01,0.001]

i = 0
treiningScore = []
predictions_score = []
precision_score =[]
recall_score = []
error_score = []
f_measure_score = []

def runModels(kernels, cPara, gamma):

    lambdaC = [atribuirTrainTest(train_index, test_index, k, c, g) for k in kernels for g in gamma for c in cPara for train_index, test_index in rkf.split(X_train)]
    return pd.DataFrame(lambdaC,columns=["K","C","Gamma","MeanScoreTraining","MeanScoreTest"])

def atribuirTrainTest(train_index, test_index, k, c, g):
    print(f"Treinando com Kernel:{k}")
    
    X_train_1, X_test_1 = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_1, y_test_1 = y_train.iloc[train_index], y_train.iloc[test_index]
    svm = SVC(kernel=k, degree=8, gamma=g,C=c,decision_function_shape='ovo').fit(X_train_1, y_train_1) 
    treiningScore = svm.fit(X_train_1,y_train_1).score(X_train_1,y_train_1)
    y_pred_1 = svm.predict(X_test_1)
    
    predictions_score.append(accuracy_score(y_pred_1, y_test_1))
    a, b , c , d = getMetrics(y_pred_1, y_test_1)
    precision_score.append(a)
    recall_score.append(b)
    error_score.append(c)
    f_measure_score.append(d)
    return [treiningScore, predictions_score
            ,np.mean(precision_score),np.mean(recall_score)
            ,np.mean(error_score),np.mean(f_measure_score)
            , svm]

df_results = runModels(kernels, C, gamma)
df_results

Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
Treinando com Kernel:rbf
